# Project

In [ ]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from types import *
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))


promo_dataset = pd.read_csv("promotion_dataset_subject.csv", encoding='utf-8')
iit_subject_dataset = pd.read_csv('iit_email_subject.csv',encoding='latin1') 
iit_subject_dataset = iit_subject_dataset.drop('Unnamed: 0',1)
promo_dataset = promo_dataset.drop('Unnamed: 0',1)
New_dataset = iit_subject_dataset.append(promo_dataset,ignore_index=True)
New_dataset['Email'] = New_dataset[['From', 'Subject']].apply(lambda x: ''.join(x), axis=1)

New_dataset=New_dataset.fillna(9)

drop_list=[]
for i in range(0,len(New_dataset['Label'])):
    if New_dataset.iloc[i]['Label']==9:
        drop_list.append(i)

New_dataset = New_dataset.drop(drop_list).reset_index(drop=True)
New_dataset.Label = New_dataset.Label.astype(int)
Y = New_dataset.iloc[:,4:5]




In [ ]:

print(New_dataset.groupby('Label').count())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#vectorizer = CountVectorizer(max_df=0.5,stop_words='english')#shape=(500, 20928) for Email
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (wnl.lemmatize(w) for w in analyzer(doc))

vectorizer = CountVectorizer(analyzer=stemmed_words,binary=True)#shape=(500, 20936) for Email
X_sparse = vectorizer.fit_transform(New_dataset['Email'])#shape=(561, 1159)
feature_name = vectorizer.get_feature_names()
X = pd.DataFrame(X_sparse.todense()) 

print("feature total number :",len(feature_name))

## Set Classifier and parameter

In [ ]:
def Random_Sampling_lwoR():
    RS_Lwo_R=[]
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.99, random_state = seed)
    for i in range(0,Budget): 
        length_trainset.append(len(X_train))
        nb.fit(X_train, Y_train)
        predicted = nb.predict(X_test)
        RS_Lwo_R.append(nb.score(X_test,Y_test))
        X_train = X_train.append(X_test.iloc[0:5])
        X_test = X_test.drop(X_test.iloc[0:5].index)
        Y_train = Y_train.append(Y_test.iloc[0:5])
        Y_test = Y_test.drop(Y_test.iloc[0:5].index)
Random_Sampling_lwoR()
print("Accuracy: ",RS_Lwo_R)

## Finding_Rationale

In [ ]:
Social_rationale_list=["linkedin","indeed","datacamp","ita"]
Primary_rationale_list=["iit","piazza","submission","handshake","collaborate","confirm","confirmation"
,"edu","application","association","apics","hawklink","comsubmission"]
promo_rationale_list=["purchase","special","off","gift","sample","shipping","promotion","left"
,"tonight","sale","new","miss","save","discount","free","last","only"
,"soon","deal","exclusive","today","ends","best","prices","weekend","holiday","now"
,"reward","final","release","clearance","collection","minute"]

Rationale_list = [Primary_rationale_list,promo_rationale_list,Social_rationale_list]
                
def findind_rationale(uncertain_index):
    print("Finding_Rationale: ")
    for i, value in enumerate(uncertain_index):
        label = Y_test.loc[value,'Label']
        for j in range(0,len(feature_name)):
            if X_test.loc[value, j] >0:
                for k,rationale in enumerate(Rationale_list[label]):       
                    if rationale == feature_name[j]:
                        print(New_dataset.iloc[value]['Email'])
                        print("Rationale:",rationale)
                        X_test.set_value(value,j,10)



## Finding_uncertain

In [ ]:
def finding_uncertain(X_test):
    print("Uncertain_Email")
    predict_proba = nb.predict_proba(X_test)
    ratio = []
    neural_index = []
    for x in range(0,len(X_test)):
        ratio.append(predict_proba[x,1]/0.3+predict_proba[x,0]/0.3+predict_proba[x,2]/0.3)
    ratio = np.array(ratio)
    neural_row = np.argsort(ratio)[::1][:5]
    for i,value in enumerate(neural_row):
        print(X_test.index[value],New_dataset.iloc[X_test.index[value]]['Email'])
        neural_index.append(X_test.index[value])
    return(neural_index)


## Random Sampling- Lw/R

In [ ]:
def Random_Sampling_lwR():
    RS_Lw_R=[]
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.99, random_state = seed)
    for i in range(0,Budget):
        print(i)
        nb.fit(X_train, Y_train)
        predicted = nb.predict(X_test)
        RS_Lw_R.append(nb.score(X_test,Y_test))
        random_list = X_test.iloc[0:5].index
        print(random_list)
        findind_rationale(random_list)
        X_train = X_train.append(X_test.iloc[0:5])
        X_test = X_test.drop(X_test.iloc[0:5].index)
        Y_train = Y_train.append(Y_test.iloc[0:5])
        Y_test = Y_test.drop(Y_test.iloc[0:5].index)

Random_Sampling_lwR()
print("Accuracy: ",RS_Lw_R)

## Active Learning- Lw/oR

In [ ]:
def Active_Learning_lwoR():
    AL_Lwo_R=[]
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.99, random_state = seed)
    for i in range(0,Budget): 
        print(i)
        nb.fit(X_train, Y_train)
        predicted = nb.predict(X_test)
        AL_Lwo_R.append(nb.score(X_test,Y_test))
        uncertain_list = finding_uncertain(X_test)
        for i, value in enumerate(uncertain_list):
            X_train = X_train.append(X_test.loc[value])
            Y_train = Y_train.append(Y_test.loc[value])
            X_test = X_test.drop(value)
            Y_test = Y_test.drop(value)
Active_Learning_lwoR()
print("Accuracy: ",AL_Lwo_R)

## Active Learning- Lw/R

In [ ]:
def Active_Learning_lwR():
    AL_Lw_R=[]
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.99, random_state = seed)
    for i in range(0,Budget):
        print(i)
        nb.fit(X_train, Y_train)
        predicted = nb.predict(X_test)
        AL_Lw_R.append(nb.score(X_test,Y_test))
        uncertain_list = finding_uncertain(X_test)
        findind_rationale(uncertain_list)
        for i, value in enumerate(uncertain_list): 
            X_train = X_train.append(X_test.loc[value])
            Y_train = Y_train.append(Y_test.loc[value])
            X_test = X_test.drop(value)
            Y_test = Y_test.drop(value)
Active_Learning_lwR()
print("Accuracy: ",AL_Lw_R)

## Compare the result Lw/R & Lw/oR -CountVectorizer